In [1]:
from glob import glob
# test_data  077
%matplotlib inline
from sklearn.metrics import confusion_matrix
import numpy as np
import pickle
from tqdm import tqdm
from collections import Counter
import os
import json
from datetime import datetime
from sklearn.model_selection import KFold
import pandas as pd
import torch
from torch import nn, optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertJapaneseTokenizer
from pytorch_pretrained_bert.optimization import BertAdam

from sklearn import metrics # https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics
from sklearn.exceptions import UndefinedMetricWarning
import warnings
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

from model import BertPunc
from data import load_file, preprocess_data, create_data_loader

In [2]:
def validate(model, criterion, epoch, epochs, iteration, iterations, data_loader_valid,
             save_path, train_loss, best_val_loss, best_model_path, punctuation_enc1, punctuation_enc2):

    val_losses = []
    val_accs_A = []
    val_accs_B = []
    val_f1s_A = []
    val_f1s_B = []
    label_keys_A = list(punctuation_enc1.keys())
    label_keys_B = list(punctuation_enc2.keys())
    label_vals_A = list(punctuation_enc1.values())
    label_vals_B = list(punctuation_enc2.values())

    #label_keys = list(punctuation_enc.keys())
    #label_vals = list(punctuation_enc.values())
    y_pred_all_A = None
    y_pred_all_B = None 
    labels_all_A = None
    labels_all_B = None

    for inputs, labelA, labelB in tqdm(data_loader_valid, total=len(data_loader_valid)):

        with torch.no_grad():

            inputs, labelA, labelB = inputs.cuda(), labelA.cuda(), labelB.cuda()
            #output = model(inputs)
            output = model(inputs)
            y_pred_A = output[0].argmax(dim=1).cpu().numpy().flatten()
            y_pred_B = output[1].argmax(dim=1).cpu().numpy().flatten()
            y_true_A = labelA.cpu().data.numpy().flatten()
            y_true_B = labelB.cpu().data.numpy().flatten()
            _loss = criterion(output[0], labelA)
            _loss += criterion(output[1], labelB)
            val_losses.append(_loss.cpu().data.numpy())
            #val_losses.append(val_loss.cpu().data.numpy())

            #y_pred = output.argmax(dim=1).cpu().data.numpy().flatten()
            #y_true = labels.cpu().data.numpy().flatten()
            
            val_accs_A.append(metrics.accuracy_score(y_true_A, y_pred_A))
            val_accs_B.append(metrics.accuracy_score(y_true_B, y_pred_B))
            val_f1s_A.append(metrics.f1_score(y_true_A, y_pred_A, average=None, labels=label_vals_A))
            val_f1s_B.append(metrics.f1_score(y_true_B, y_pred_B, average=None, labels=label_vals_B))
            
    
    val_loss = np.mean(val_losses)
    val_acc_A = np.mean(val_accs_A)
    val_acc_B = np.mean(val_accs_B)
    val_f1_A = np.array(val_f1s_A).mean(axis=0)
    val_f1_B = np.array(val_f1s_B).mean(axis=0)

    improved = ''

    # model_path = '{}model_{:02d}{:02d}'.format(save_path, epoch, iteration)
    model_path = save_path+'model'
    torch.save(model.state_dict(), model_path)
    if val_loss < best_val_loss:
        improved = '*'
        best_val_loss = val_loss
        best_model_path = model_path

    f1_cols_A = ';'.join(['f1_'+key for key in label_keys_A])
    f1_cols_B = ';'.join(['f1_'+key for key in label_keys_B])

    progress_path = save_path+'progress.csv'
    if not os.path.isfile(progress_path):
        with open(progress_path, 'w') as f:
            f.write('time;epoch;iteration;training loss;loss;accuracy_A;accuracy_B;'+f1_cols_A+f1_cols_B+'\n')

    f1_vals_A = ';'.join(['{:.4f}'.format(val) for val in val_f1_A])
    f1_vals_B = ';'.join(['{:.4f}'.format(val) for val in val_f1_B])
    #f1_As = ';'.join(['{:.4f}'.format(f1_A)])
    #f1_Bs = ';'.join(['{:.4f}'.format(f1_B)])

    with open(progress_path, 'a',encoding='utf-8') as f:
        f.write('{};{};{};{:.4f};{:.4f};{:.4f};{:.4f};{};{}\n'.format(
            datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            epoch+1,
            iteration,
            train_loss,
            val_loss,
            val_acc_A,
            val_acc_B,
            f1_vals_A,
            f1_vals_B
            ))

    print("Epoch: {}/{}".format(epoch+1, epochs),
          "Iteration: {}/{}".format(iteration, iterations),
          "Loss: {:.4f}".format(train_loss),
          "Val Loss: {:.4f}".format(val_loss),
          "Acc for taskA: {:.4f}".format(val_acc_A),
          "Acc for taskB: {:.4f}".format(val_acc_B),
          "F1 for taskA: {}".format(f1_vals_A),
          "F1 for taskB: {}".format(f1_vals_B),
          improved)

    return best_val_loss, best_model_path

In [3]:
def train(model, optimizer, criterion, epochs, data_loader_train, data_loader_valid, 
          save_path,punctuation_enc1, punctuation_enc2, iterations=3, best_val_loss=1e9):
    #y_pred_all_A = None
    #y_pred_all_B = None 
    #labels_all_A = None
    #labels_all_B = None

    print_every = len(data_loader_train)//iterations+1
    clip = 5
    best_model_path = None
    model.train()
    pbar = tqdm(total=print_every)

    for e in range(epochs):

        counter = 1
        iteration = 1

        for inputs, labelA, labelB in data_loader_train:

            inputs, labelA, labelB = inputs.cuda(), labelA.cuda(), labelB.cuda()
            inputs.requires_grad = False
            labelA.requires_grad = False
            labelB.requires_grad = False
            output = model(inputs)
            loss = criterion(output[0], labelA)
            loss += criterion(output[1], labelB)
            #loss = _loss.item()
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            train_loss = loss.cpu().data.numpy()
            
            pbar.update()
            
            if counter % print_every == 0:
                
                pbar.close()
                model.eval()
                best_val_loss, best_model_path = validate(model, criterion, e, epochs, iteration, iterations, data_loader_valid, 
                    save_path, train_loss, best_val_loss, best_model_path, punctuation_enc1, punctuation_enc2)
                model.train()
                pbar = tqdm(total=print_every)
                iteration += 1

            counter += 1

        pbar.close()
        model.eval()
        best_val_loss, best_model_path = validate(model, criterion, e, epochs, iteration, iterations, data_loader_valid, 
            save_path, train_loss, best_val_loss, best_model_path, punctuation_enc1, punctuation_enc2)
        model.train()
        if e < epochs-1:
            pbar = tqdm(total=print_every)
                
    model.load_state_dict(torch.load(best_model_path))
    model.eval()

    return model, optimizer, best_val_loss


In [4]:
punctuation_enc1 = {
    'O': 0,
    'COMMA': 1
}#task1 for comma 

punctuation_enc2 = {
    'O': 0,
    'L': 1
}#task2 for lf
segment_size = 16
dropout = 0.3
epochs_top = 3
iterations_top = 2
batch_size_top = 64
learning_rate_top = 1e-5
epochs_all = 5
iterations_all = 3
batch_size_all = 32
learning_rate_all = 1e-5
hyperparameters = {
     'segment_size': segment_size,
     'dropout': dropout,
     'epochs_top': epochs_top,
     'iterations_top': iterations_top,
     'batch_size_top': batch_size_top,
     'learning_rate_top': learning_rate_top,
     'epochs_all': epochs_all,
     'iterations_all': iterations_all,
     'batch_size_all': batch_size_all,
    'learning_rate_all': learning_rate_all,
}
save_path = 'models/{}/'.format(datetime.now().strftime("%Y%m%d_%H%M%S"))
os.mkdir(save_path)
with open(save_path+'hyperparameters.json', 'w') as f:
    json.dump(hyperparameters, f)

print('LOADING DATA...')
# test 1 
# data_train = load_file('data/LREC/train2012')
# data_valid = load_file('data/LREC/dev2012')
# data_test = load_file('data/LREC/test2011')
# data_test_asr = load_file('data/LREC/test2011asr')
#data_train = load_file('D:/bertpunc-newplan/data/train')

data_valid_task1 = load_file('data/comma/SXOPX037.txt')
data_valid_task1 += load_file('data/comma/SXOPX062.txt')

data_valid_task2 = load_file('data/lf/SXOPX037.txt')
data_valid_task2 += load_file('data/lf/SXOPX062.txt')
#print(data_valid)
#data_test = load_file('D:/bertpunc-newplan2/data/test')
#data_test_task1 = load_file('data/comma/SXOPX003.txt')

#data_test_task2 = load_file('data/lf/SXOPX003.txt')


data_train_task1 = load_file('data/comma/SXOPX003.txt')
data_train_task1 += load_file('data/comma/SXOPX098.txt')
data_train_task1 += load_file('data/comma/SXPPX017.txt')
data_train_task1 += load_file('data/comma/SXRPX045.txt')
data_train_task1 += load_file('data/comma/SXRPX048.txt')
data_train_task1 += load_file('data/comma/SXRPX054.txt')
data_train_task1 += load_file('data/comma/SXRPX083.txt')
data_train_task1 += load_file('data/comma/SXRPX104.txt')
data_train_task1 += load_file('data/comma/SXRSX027.txt')
data_train_task1 += load_file('data/comma/SXRSX070.txt')
data_train_task1 += load_file('data/comma/SXTPX024.txt')
data_train_task1 += load_file('data/comma/SXUPX011.txt')
data_train_task1 += load_file('data/comma/SXVPX034.txt')


data_train_task2 = load_file('data/lf/SXOPX003.txt')
data_train_task2 += load_file('data/lf/SXOPX098.txt')
data_train_task2 += load_file('data/lf/SXPPX017.txt')
data_train_task2 += load_file('data/lf/SXRPX045.txt')
data_train_task2 += load_file('data/lf/SXRPX048.txt')
data_train_task2 += load_file('data/lf/SXRPX054.txt')
data_train_task2 += load_file('data/lf/SXRPX083.txt')
data_train_task2 += load_file('data/lf/SXRPX104.txt')
data_train_task2 += load_file('data/lf/SXRSX027.txt')
data_train_task2 += load_file('data/lf/SXRSX070.txt')
data_train_task2 += load_file('data/lf/SXTPX024.txt')
data_train_task2 += load_file('data/lf/SXUPX011.txt')
data_train_task2 += load_file('data/lf/SXVPX034.txt')

#data_trainandtest = load_file('D:/bertpunc-newplan2/data/trainandtest')
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese')
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#kf = KFold()
print('PREPROCESSING DATA...')
#X_trainandtest, y_trainandtest  = preprocess_data(data_trainandtest, tokenizer, punctuation_enc, segment_size)
#X_train, X_test, y_train, y_test = train_test_split(X_trainandtest, y_trainandtest, test_size=0.14, random_state=42)
X_train, y1_train = preprocess_data(data_train_task1, tokenizer, punctuation_enc1, segment_size)

X_train, y2_train = preprocess_data(data_train_task2, tokenizer, punctuation_enc2, segment_size)

X_valid, y1_valid = preprocess_data(data_valid_task1, tokenizer, punctuation_enc1, segment_size)

X_valid, y2_valid = preprocess_data(data_valid_task2, tokenizer, punctuation_enc2, segment_size)

#X_test, y1_test = preprocess_data(data_test_task1, tokenizer, punctuation_enc1, segment_size)

#X_test, y2_test = preprocess_data(data_test_task2, tokenizer, punctuation_enc2, segment_size)
#print(X_valid)
#print(y_valid)
print('INITIALIZING MODEL...')
output_size1 = len(punctuation_enc1)
output_size2 = len(punctuation_enc2)
bert_punc = nn.DataParallel(BertPunc(segment_size, output_size1, output_size2, dropout).cuda())





LOADING DATA...
PREPROCESSING DATA...
INITIALIZING MODEL...


Some weights of the model checkpoint at cl-tohoku/bert-base-japanese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
print('TRAINING TOP LAYER...')
data_loader_train = create_data_loader(X_train, y1_train, y2_train, True, batch_size_top)
data_loader_valid = create_data_loader(X_valid, y1_valid, y2_valid, False, batch_size_top)
for p in bert_punc.module.bert.parameters():
    p.requires_grad = False
optimizer = optim.Adam(bert_punc.parameters(), lr=learning_rate_top)
criterion = nn.CrossEntropyLoss()
bert_punc, optimizer, best_val_loss = train(bert_punc, optimizer, criterion, epochs_top, 
    data_loader_train, data_loader_valid, save_path, punctuation_enc1, punctuation_enc2, iterations_top, best_val_loss=1e9)



TRAINING TOP LAYER...


100%|██████████| 144/144 [00:04<00:00, 34.98it/s]


Epoch: 1/3 Iteration: 1/2 Loss: 0.4022 Val Loss: 0.4677 Acc for taskA: 0.9376 Acc for taskB: 0.9150 F1 for taskA: 0.9658;0.6122 F1 for taskB: 0.9521;0.5935 *


100%|██████████| 144/144 [00:04<00:00, 34.76it/s]


Epoch: 1/3 Iteration: 2/2 Loss: 0.3935 Val Loss: 0.2978 Acc for taskA: 0.9480 Acc for taskB: 0.9446 F1 for taskA: 0.9718;0.6344 F1 for taskB: 0.9695;0.6813 *


...

Epoch: 2/3 Iteration: 2/2 Loss: 0.1637 Val Loss: 0.3086 Acc for taskA: 0.9471 Acc for taskB: 0.9405 F1 for taskA: 0.9711;0.6507 F1 for taskB: 0.9670;0.6746 


100%|██████████| 144/144 [00:04<00:00, 34.96it/s]


Epoch: 3/3 Iteration: 1/2 Loss: 0.0578 Val Loss: 0.2853 Acc for taskA: 0.9468 Acc for taskB: 0.9485 F1 for taskA: 0.9712;0.6213 F1 for taskB: 0.9718;0.6890 *


100%|██████████| 144/144 [00:04<00:00, 35.42it/s]


Epoch: 3/3 Iteration: 2/2 Loss: 0.2391 Val Loss: 0.3150 Acc for taskA: 0.9470 Acc for taskB: 0.9482 F1 for taskA: 0.9714;0.6069 F1 for taskB: 0.9716;0.6847 


In [6]:
print('TRAINING ALL LAYERS...')
data_loader_train = create_data_loader(X_train, y1_train, y2_train, True, batch_size_all)
data_loader_valid = create_data_loader(X_valid, y1_valid, y2_valid, False, batch_size_all)
for p in bert_punc.module.bert.parameters():
    p.requires_grad = True
optimizer = optim.Adam(bert_punc.parameters(), lr=learning_rate_all)
criterion = nn.CrossEntropyLoss()
bert_punc, optimizer, best_val_loss = train(bert_punc, optimizer, criterion, epochs_all, 
    data_loader_train, data_loader_valid, save_path, punctuation_enc1, punctuation_enc2, iterations_all, best_val_loss=best_val_loss)

TRAINING ALL LAYERS...


100%|██████████| 288/288 [00:04<00:00, 59.74it/s]


Epoch: 1/5 Iteration: 1/3 Loss: 0.1334 Val Loss: 0.3485 Acc for taskA: 0.9515 Acc for taskB: 0.9451 F1 for taskA: 0.9733;0.6504 F1 for taskB: 0.9694;0.6759 


100%|██████████| 288/288 [00:05<00:00, 56.36it/s]


Epoch: 1/5 Iteration: 2/3 Loss: 0.5285 Val Loss: 0.3219 Acc for taskA: 0.9504 Acc for taskB: 0.9427 F1 for taskA: 0.9729;0.6184 F1 for taskB: 0.9686;0.6081 


...

Epoch: 5/5 Iteration: 1/3 Loss: 0.1451 Val Loss: 0.2714 Acc for taskA: 0.9621 Acc for taskB: 0.9589 F1 for taskA: 0.9793;0.6910 F1 for taskB: 0.9772;0.7583 


100%|██████████| 288/288 [00:04<00:00, 58.77it/s]


Epoch: 5/5 Iteration: 2/3 Loss: 0.0851 Val Loss: 0.3206 Acc for taskA: 0.9581 Acc for taskB: 0.9595 F1 for taskA: 0.9772;0.6473 F1 for taskB: 0.9780;0.6953 


100%|██████████| 288/288 [00:04<00:00, 58.05it/s]


Epoch: 5/5 Iteration: 3/3 Loss: 0.0701 Val Loss: 0.2386 Acc for taskA: 0.9642 Acc for taskB: 0.9603 F1 for taskA: 0.9803;0.7260 F1 for taskB: 0.9783;0.7304 


In [7]:
sorted(glob('models/*'))  # newplan2's models are in newplan's

['models/20220905_201531', 'models/20220905_203404']

In [8]:
path = 'models/20220905_203404/'

In [9]:
with open(path+'hyperparameters.json', 'r') as f:
    hyperparameters = json.load(f)
hyperparameters

{'segment_size': 16,
 'dropout': 0.3,
 'epochs_top': 3,
 'iterations_top': 2,
 'batch_size_top': 64,
 'learning_rate_top': 1e-05,
 'epochs_all': 5,
 'iterations_all': 3,
 'batch_size_all': 32,
 'learning_rate_all': 1e-05}

In [10]:
#tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese')

punctuation_enc1 = {
    'O': 0,
    'COMMA': 1
}
punctuation_enc2 = {
    'O': 0,
    'L': 1
}

# punctuation_enc = {
#     'O': 0,
#     'PERIOD': 1
# }

segment_size = hyperparameters['segment_size']

In [17]:
output_size = len(punctuation_enc1)
dropout = hyperparameters['dropout']
bert_punc = nn.DataParallel(BertPunc(segment_size, output_size, dropout).cuda())

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
progress = pd.read_csv(path+'progress.csv', delimiter=';')
progress

,time,epoch,iteration,training loss,loss,accuracy_A,accuracy_B,f1_O,f1_COMMAf1_O,f1_L
2022-09-05 20:34:53,1,1,0.4022,0.4677,0.9376,0.9150,0.9658,0.6122,0.9521,0.5935
2022-09-05 20:35:13,1,2,0.3935,0.2978,0.9480,0.9446,0.9718,0.6344,0.9695,0.6813
2022-09-05 20:35:34,2,1,0.1926,0.2872,0.9463,0.9406,0.9707,0.6399,0.9673,0.6581
2022-09-05 20:35:54,2,2,0.1637,0.3086,0.9471,0.9405,0.9711,0.6507,0.9670,0.6746
2022-09-05 20:36:14,3,1,0.0578,0.2853,0.9468,0.9485,0.9712,0.6213,0.9718,0.6890
2022-09-05 20:36:34,3,2,0.2391,0.3150,0.9470,0.9482,0.9714,0.6069,0.9716,0.6847
2022-09-05 20:37:20,1,1,0.1334,0.3485,0.9515,0.9451,0.9733,0.6504,0.9694,0.6759
2022-09-05 20:38:09,1,2,0.5285,0.3219,0.9504,0.9427,0.9729,0.6184,0.9686,0.6081
2022-09-05 20:38:58,1,3,0.0108,0.2350,0.9537,0.9572,0.9746,0.6618,0.9764,0.7313
2022-09-05 20:39:52,2,1,0.0736,0.2417,0.9554,0.9570,0.9756,0.6443,0.9765,0.7157


In [12]:
bert_punc.load_state_dict(torch.load(path+'model'))
bert_punc.eval();

In [13]:
def predictions(data_loader):
    y1_pred = []
    y2_pred = []
    y1_true = []
    y2_true = []

    for inputs, labelA, labelB in tqdm(data_loader, total=len(data_loader)):
        with torch.no_grad():
            inputs, labelA, labelB = inputs.cuda(), labelA.cuda(), labelB.cuda()
            output = bert_punc(inputs)
            y1_pred += list(output[0].argmax(dim=1).cpu().data.numpy().flatten())
            y2_pred += list(output[1].argmax(dim=1).cpu().data.numpy().flatten())
            y1_true += list(labelA.cpu().data.numpy().flatten())
            y2_true += list(labelB.cpu().data.numpy().flatten())
    print('PREDICATION for task1: comma insert')
    print(y1_pred)
    print('TRUE ANSWER for task1: comma insert')
    print(y1_true)
    print('PREDICATION for task2: linefeed insert')
    print(y2_pred)
    print('TRUE ANSWER for task2: linefeed insert')
    print(y2_true)
        
    return y1_pred, y2_pred, y1_true, y2_true

In [14]:
def evaluation(y_pred, y_true, punctuation_enc):
    f = confusion_matrix(y_true, y_pred)
    print(f)
    precision, recall, f1, _ = metrics.precision_recall_fscore_support(
        y_true, y_pred, average=None, labels=[1])
    #overall = metrics.precision_recall_fscore_support(
        #y_true, y_pred, average='macro', labels=[1])
    result = pd.DataFrame(
        np.array([precision, recall, f1]), 
        columns=list(punctuation_enc.keys())[1:], 
        index=['Precision', 'Recall', 'F1']
    )
    #result['OVERALL'] = overall[:3]
    return result

In [15]:
batch_size = 16
data_test_task1 = glob('data/comma/077/*')
#data_test_task2 = glob('data/lf/SXOPX003/*')
y1_pred = []
y2_pred = []
y1_true = []
y2_true = []
count = 0
for data in data_test_task1:
    count += 1
for i in range(1, count+1):
    print(str(i)+'.txt')
    filename_task1 = 'data/comma/077/'+str(i)+'.txt'
    filename_task2 = 'data/lf/077/'+str(i)+'.txt'
    data_test_task1 = load_file(filename_task1)
    data_test_task2 = load_file(filename_task2)
    X_test, y1_test = preprocess_data(data_test_task1, tokenizer, punctuation_enc1, segment_size)
    X_test, y2_test = preprocess_data(data_test_task2, tokenizer, punctuation_enc2, segment_size)
    data_loader_test = create_data_loader(X_test, y1_test, y2_test, False, batch_size)
    y1_pred_test, y2_pred_test, y1_true_test, y2_true_test = predictions(data_loader_test)
    y1_pred += y1_pred_test
    y1_true += y1_true_test
    y2_pred += y2_pred_test
    y2_true += y2_true_test

#data_loader_test = create_data_loader(X_test, y1_test, y2_test, False, batch_size)
#data_loader_test_asr = create_data_loader(X_test_asr, y_test_asr, False, batch_size)

#y1_pred_test, y2_pred_test, y1_true_test, y2_true_test = predictions(data_loader_test)
#print(y1_pred_test, y2_pred_test, y1_true_test, y2_true_test)
print('evaluation for task1: comma insert')
eval1_test = evaluation(y1_pred, y1_true, punctuation_enc1)
print(eval1_test)
print('\n')
print('evaluation for task2: linefeed insert')
eval2_test = evaluation(y2_pred, y2_true, punctuation_enc2)
print(eval2_test)


1.txt


100%|██████████| 2/2 [00:00<00:00, 52.38it/s]


PREDICATION for task1: comma insert
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
TRUE ANSWER for task1: comma insert
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
PREDICATION for task2: linefeed insert
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
TRUE ANSWER for task2: linefeed insert
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
2.txt


100%|██████████| 3/3 [00:00<00:00, 50.32it/s]


PREDICATION for task1: comma insert
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
TRUE ANSWER for task1: comma insert
[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
PREDICATION for task2: linefeed insert
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
TRUE ANSWER for task2: linefeed insert
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
3.txt


...

PREDICATION for task1: comma insert
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
TRUE ANSWER for task1: comma insert
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
PREDICATION for task2: linefeed insert
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
TRUE ANSWER for task2: linefeed insert
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
99.txt


100%|██████████| 3/3 [00:00<00:00, 80.12it/s]


PREDICATION for task1: comma insert
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
TRUE ANSWER for task1: comma insert
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
PREDICATION for task2: linefeed insert
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
TRUE ANSWER for task2: linefeed insert
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
100.txt


100%|██████████| 2/2 [00:00<00:00, 75.19it/s]

PREDICATION for task1: comma insert
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
TRUE ANSWER for task1: comma insert
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
PREDICATION for task2: linefeed insert
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
TRUE ANSWER for task2: linefeed insert
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
evaluation for task1: comma insert
[[4202   45]
 [  60  371]]
              COMMA
Precision  0.891827
Recall     0.860789
F1         0.876033


evaluation for task2: linefeed insert
[[4236   56]
 [ 103  283]]
                  L
Precision  0.834808
Recall     0.733161
F1         0.780690
